## Notebook Lección 7


### Objetivos Lección 7 y 8

1) Implementar el modelo de Media-Varianza de Markowitz con y sin restricciones.

2) Entender las generalidad del modelo de Black Litterman y usar el método de Idzorek para computarlo

### Ejercicio

1. Estime la matriz de varianza de los activos con el método EWMA.
2. Estime la matriz de varianza de los activos con el método histórico.
4. Estime la distribución de retornos apriori. 
5. Corra el ejercicio de Black Litterman con el view "El retorno de la clase de activo que proyectamos en el horizonte de inversión es de un 8.5% para equity y un 4% para fixed income, se tienen niveles de confianza respectivamente del 50% y 15%"(use la metodología de Idzorek).  
6. Ahora asuma que se cumple por completo la eficiencia del mercado y que por tanto tau es cero. Genere la frontera eficiente. Use EWMA en su estimación.
7. Encuentre el portafolio de máximo Sharpe-Ratio. 

In [15]:
import numpy as np
import pandas as pd
days_in_year = 252
window = 21

In [16]:
assets_df = pd.read_csv("FIEquity.csv", encoding='utf-8', sep = ";", header = [3,5], index_col= 0)
#assets_df.columns = assets_df.iloc[0,:]
#assets_df = assets_df.dropna(how ='all' )
#assets_df = assets_df.iloc[2:,:]
#assets_df.index = assets_df['Dates']
#assets_df = assets_df.iloc[:,1:]
#assets_df.index = pd.to_datetime(assets_df['Dates'])
dict_columns = {'Unnamed: 0_level_0': ' ','Unnamed: 2_level_0': 'B500 Index', 'Unnamed: 4_level_0': 'LBUSTRUU Index' }
assets_df = assets_df.rename(dict_columns ,axis =1, level = 0)
assets_df.index = pd.to_datetime(assets_df.index)
assets_df
idx = pd.IndexSlice

# Función que estima la volatilidad con el método EWMA
def calculateEWMAVol(log_returns, sigma_seed, lambda_scalar):
    # Calculate the 
    log_returns_squared = log_returns**2
    log_returns_squared = log_returns_squared.dropna()
    log_returns_squared.iloc[0] = sigma_seed**2
    log_returns_squared = log_returns_squared.ewm(alpha = 1-lambda_scalar, adjust= False).mean()
    log_returns_squared = np.sqrt(log_returns_squared)
    return log_returns_squared
assets_df.loc[:,idx['B500 Index','Log_Returns']] = np.log((assets_df.loc[:,idx['B500 Index','PX_LAST']].shift(-1)/assets_df.loc[:,idx['B500 Index','PX_LAST']]).shift(1))
assets_df.loc[:,idx['LBUSTRUU Index','Log_Returns']] = np.log((assets_df.loc[:,idx['LBUSTRUU Index','PX_LAST']].shift(-1)/assets_df.loc[:,idx['LBUSTRUU Index','PX_LAST']]).shift(1))
assets_df.loc[:,idx['B500 Index','Sd']] = assets_df.loc[:,idx['B500 Index','Log_Returns']].rolling(window=window).apply(np.std, raw=True)
assets_df.loc[:,idx['LBUSTRUU Index','Sd']] = assets_df.loc[:,idx['LBUSTRUU Index','Log_Returns']].rolling(window=window).apply(np.std, raw=True)

cov = assets_df.loc[:,idx[:,'Log_Returns']].rolling(window=window).cov()
cov = cov.dropna()
assets_df = assets_df.sort_index(axis = 1)
assets_df = assets_df.dropna()
assets_df

<ipython-input-16-c5d2132ed317>:10: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  assets_df.index = pd.to_datetime(assets_df.index)


B500 Index                                 LBUSTRUU Index  \
Dates      INDX_MARKET_CAP Log_Returns  PX_LAST        Sd INDX_MARKET_CAP   
2008-12-31      7868850.08    0.008824   322.14  0.055420     11430264.42   
2009-01-30      7211885.39   -0.086114   295.56  0.055243     11448304.34   
2009-02-27      6434213.30   -0.113264   263.91  0.056432     11466836.26   
2009-03-31      7056934.17    0.081991   286.46  0.061562     11769390.95   
2009-04-30      7676108.30    0.086109   312.22  0.066287     11985155.30   
...                    ...         ...      ...       ...             ...   
2022-09-30     30198312.02   -0.098631  1286.04  0.051885     23970316.29   
2022-10-31     32495633.95    0.074811  1385.94  0.054453     23756461.86   
2022-11-30     34151554.37    0.049829  1456.75  0.055262     24639812.54   
2022-12-30     32000410.19   -0.061864  1369.36  0.056301     24541031.29   
2023-01-31     34023584.90    0.061308  1455.94  0.056779     25424595.64   

                                           
Dates      Log_Returns  PX_LAST        Sd  
2008-12-31    0.036630  1454.10  0.013499  
2009-01-30   -0.008862  1441.27  0.013814  
2009-02-27   -0.003782  1435.83  0.013681  
2009-03-31    0.013806  1455.79  0.013715  
2009-04-30    0.004770  1462.75  0.013698  
...                ...      ...       ...  
2022-09-30   -0.044170  2011.06  0.016712  
2022-10-31   -0.013038  1985.01  0.016741  
2022-11-30    0.036116  2058.01  0.019175  
2022-12-30   -0.004519  2048.73  0.019123  
2023-01-31    0.030302  2111.76  0.020435  

[170 rows x 8 columns]

In [17]:
cov = cov.loc[idx[:,:,'Log_Returns'],idx['B500 Index','Log_Returns']].reset_index(level = 1, drop = True).reset_index(level = 1, drop = True).rename('Covarianza')
cross_returns = assets_df.loc[:,idx['B500 Index','Log_Returns']]*assets_df.loc[:,idx['LBUSTRUU Index','Log_Returns']]
cross_returns

2008-12-31    0.000323
2009-01-30    0.000763
2009-02-27    0.000428
2009-03-31    0.001132
2009-04-30    0.000411
                ...   
2022-09-30    0.004357
2022-10-31   -0.000975
2022-11-30    0.001800
2022-12-30    0.000280
2023-01-31    0.001858
Length: 170, dtype: float64

In [18]:
def calculateEWMACov(log_returns, sigma_seed, lambda_scalar):
    # Calculate the 
    log_returns = log_returns.dropna()
    log_returns.iloc[0] = sigma_seed
    log_returns = log_returns.ewm(alpha = 1-lambda_scalar, adjust= False).mean()
    return log_returns

assets_df.loc[:,idx['B500 Index','Sd (ewma)']] = calculateEWMAVol(assets_df.loc[:,idx['B500 Index','Log_Returns']], assets_df.loc[:,idx['B500 Index','Sd']].iloc[0], 0.94)
assets_df.loc[:,idx['LBUSTRUU Index','Sd (ewma)']] = calculateEWMAVol(assets_df.loc[:,idx['LBUSTRUU Index','Log_Returns']], assets_df.loc[:,idx['LBUSTRUU Index','Sd']].iloc[0], 0.94)
cov_ewma = calculateEWMACov(cross_returns, cov.iloc[0], 0.94)
cov_ewma 

2008-12-31    0.003225
2009-01-30    0.003077
2009-02-27    0.002918
2009-03-31    0.002811
2009-04-30    0.002667
                ...   
2022-09-30    0.000743
2022-10-31    0.000640
2022-11-30    0.000710
2022-12-30    0.000684
2023-01-31    0.000754
Length: 170, dtype: float64

In [19]:
anual_variance_FI = (assets_df.loc[:,idx['LBUSTRUU Index','Sd']].iloc[-1]**2)
anual_variance_Equity = (assets_df.loc[:,idx['B500 Index','Sd']].iloc[-1]**2)
anual_covariance = (cov.iloc[-1])*252
covariance = np.array([[anual_variance_FI,anual_covariance],[anual_covariance,anual_variance_Equity]])
covariance

array([[0.00041758, 0.21271237],
       [0.21271237, 0.00322388]])

In [20]:
anual_variance_FI = (assets_df.loc[:,idx['LBUSTRUU Index','Sd (ewma)']].iloc[-1]**2)
anual_variance_Equity = (assets_df.loc[:,idx['B500 Index','Sd (ewma)']].iloc[-1]**2)
anual_covariance_ewma = (cov_ewma.iloc[-1])
covariance_ewma = np.array([[anual_variance_FI,anual_covariance_ewma],[anual_covariance_ewma,anual_variance_Equity]])
covariance_ewma = pd.DataFrame([[anual_variance_FI,anual_covariance_ewma],[anual_covariance_ewma,anual_variance_Equity]],columns=['Bonds', 'Equity'], index=['Bonds', 'Equity'])
covariance_ewma

,Bonds,Equity
Bonds,0.000426,0.000754
Equity,0.000754,0.003524


In [21]:
assets_df.loc[:,idx[:,'Log_Returns']].sum()

                Dates      
B500 Index      Log_Returns    1.517245
LBUSTRUU Index  Log_Returns    0.409765
dtype: float64

### Distribución de Retornos A Priori

Lo primero que necesitamos es encontrar el parámetro de la aversión al riesgo que se usará en la optimización en reversa. Recordemos que la fórmula es: 

$$\delta = \frac{R-R_{f}}{\sigma^2}$$

In [22]:
from pypfopt import black_litterman
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier
import yfinance as yf

tau = 0.05
snp = yf.Ticker("^GSPC")
hist_snp = snp.history(period="max")
hist_snp.index = hist_snp.index.date
hist_snp.index = pd.to_datetime(hist_snp.index)
hist_snp
delta = black_litterman.market_implied_risk_aversion(assets_df.loc[:,idx['B500 Index','PX_LAST']])
delta

4.963405428384375

### Prior Returns

Usando la ecuación que derivamos:

$$\Pi = \delta\Sigma \omega_{mkt}$$


In [23]:
mcaps = assets_df.loc[:,idx[:,'INDX_MARKET_CAP']].iloc[-1].reset_index(level=1, drop = True)
mcaps = mcaps.rename({'B500 Index':'Equity','LBUSTRUU Index': 'Bonds'})
mcaps = mcaps.sort_index()
mcaps

Bonds     25424595.64
Equity    34023584.90
Name: 2023-01-31 00:00:00, dtype: float64

In [24]:
prior = black_litterman.market_implied_prior_returns(mcaps, delta, covariance_ewma)
prior

Bonds     0.023047
Equity    0.031613
dtype: float64

In [25]:
# Market Cap 
Q = np.array([0.08]).reshape(-1, 1)
bl = BlackLittermanModel(covariance_ewma, pi = prior, Q = Q, absolute_views={'Bonds':0.04,'Equity':0.85}, view_confidences = [0.5, 0.15], omega = "idzorek")
bl.bl_weights(delta)
weights = bl.clean_weights()
weights

OrderedDict([('Bonds', 0.61854), ('Equity', 0.38146)])

In [26]:
from pypfopt import EfficientFrontier
ef = EfficientFrontier(prior, covariance_ewma)
ef.min_volatility()

OrderedDict([('Bonds', 1.0000000000051659), ('Equity', 2.414e-13)])

In [27]:
ef = EfficientFrontier(prior, covariance_ewma)
ef.max_sharpe(risk_free_rate=0.02)

OrderedDict([('Bonds', 0.4276765984614514), ('Equity', 0.5723234015385487)])

In [28]:
ef = EfficientFrontier([0.04, 0.085], covariance_ewma)
ef.max_sharpe(risk_free_rate=0.02)

OrderedDict([('Bonds', 0.6297872677582479), ('Equity', 0.370212732241752)])